In [118]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow import keras
from PIL import Image
from collections import Counter

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F


In [119]:
import xmltodict
import cv2
from PIL import Image

img_width = 224
img_height = 224

def zoom_at(img, x, y, zoom):
    w, h = img.size
    zoom2 = zoom * 2
    img = img.crop((x - w / zoom2, y - h / zoom2, 
                    x + w / zoom2, y + h / zoom2))
    return img.resize((w, h), Image.LANCZOS)

def get_number_of_positives_and_negatives(past, seq):
	dir_imgs = past + str(seq) + "/imgs/"
	positives = 0
	negatives = 0

	with open(past + str(seq) + "/annotations.xml", "rb") as org_file:
		dict_file = xmltodict.parse(org_file)

	arr_plants = dict_file["annotations"]["image"]

	for file in arr_plants:
		try:
			name_img = file["@name"]
			x_min = file["box"]
			positives += 1

		except Exception as e:
			negatives += 1
	return positives, negatives


def get_arrays_from_past_and_seq(past, seq):
	x_array = []
	y_array = []
	dir_imgs = past + str(seq) + "/imgs/"

	with open(past + str(seq) + "/annotations.xml", "rb") as org_file:
		dict_file = xmltodict.parse(org_file)

	arr_plants = dict_file["annotations"]["image"]

	for file in arr_plants:
		try:
			name_img = file["@name"]
			x_min = file["box"]
			img = Image.open(dir_imgs + name_img)
			img = img.resize((img_width, img_height)) # 0.3 = 576x324 --- 0.2 = 384x216 --- 0.15 = 288x162
			x_array.append(img)
			y_array.append(1)

		except:
			name_img = file["@name"]
			img = Image.open(dir_imgs + name_img)
			img = img.resize((img_width, img_height))
			#rotated_image1 = img.rotate(180)
			#rotated_image1 = zoom_at(rotated_image1, 960, 540, 1.5)
			# rotated_image1 = rotated_image1.resize((img_width, img_height))
			# img.show()
			# rotated_image1.show()
			#x_array.append(rotated_image1)
			#y_array.append([0])
			x_array.append(img)
			y_array.append(0)

	return x_array, y_array

past_and_last_seq_0 = ("RumexWeeds/20210806_hegnstrup/seq", 17)
past_and_last_seq_1 = ("RumexWeeds/20210806_stengard/seq", 20)
past_and_last_seq_2 = ("RumexWeeds/20210807_lundholm/seq", 28)
past_and_last_seq_3 = ("RumexWeeds/20210908_lundholm/seq", 13)
past_and_last_seq_4 = ("RumexWeeds/20211006_stengard/seq", 15)

x_train = []
y_train = []

x_test = []
y_test = []

# total_pos = 0
# total_neg = 0
# for i in range(past_and_last_seq_4[1]+1):
# 	p, n = get_number_of_positives_and_negatives(past_and_last_seq_4[0], i)S
# 	total_pos += p
# 	total_neg += n
# 	print(i, " = ", p, "  ", n)

# print(past_and_last_seq_4[0])
# print("Positives: ", total_pos)
# print("Negatives: ", total_neg, "\n")

for i in range(past_and_last_seq_2[1]+1):
	x_aux, y_aux = get_arrays_from_past_and_seq(past_and_last_seq_2[0], i)
	x_train.extend(x_aux)
	y_train.extend(y_aux)

for i in range(past_and_last_seq_0[1]+1):
	x_aux, y_aux = get_arrays_from_past_and_seq(past_and_last_seq_0[0], i)
	x_test.extend(x_aux)
	y_test.extend(y_aux)

x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

# 17 positivas
# 10 negativas SEQ0

print(x_train.shape)
print(x_test.shape)

print("Total treino = ", len(y_train))
print("Total teste = ", len(y_test))
# (580, 1200, 1920, 3)

(1128, 224, 224, 3)
(580, 224, 224, 3)
Total treino =  1128
Total teste =  580


In [120]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from keras.layers import concatenate

In [121]:

def Inception_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4): 
  # Input: 
  # - f1: number of filters of the 1x1 convolutional layer in the first path
  # - f2_conv1, f2_conv3 are number of filters corresponding to the 1x1 and 3x3 convolutional layers in the second path
  # - f3_conv1, f3_conv5 are the number of filters corresponding to the 1x1 and 5x5  convolutional layer in the third path
  # - f4: number of filters of the 1x1 convolutional layer in the fourth path

  # 1st path:
  path1 = Conv2D(filters=f1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)

  # 2nd path
  path2 = Conv2D(filters = f2_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path2 = Conv2D(filters = f2_conv3, kernel_size = (3,3), padding = 'same', activation = 'relu')(path2)

  # 3rd path
  path3 = Conv2D(filters = f3_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path3 = Conv2D(filters = f3_conv5, kernel_size = (5,5), padding = 'same', activation = 'relu')(path3)

  # 4th path
  path4 = MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
  path4 = Conv2D(filters = f4, kernel_size = (1,1), padding = 'same', activation = 'relu')(path4)

  output_layer = concatenate([path1, path2, path3, path4], axis = -1)

  return output_layer

def GoogLeNet():
  # input layer 
  input_layer = Input(shape = (img_width, img_height, 3))

  # convolutional layer: filters = 64, kernel_size = (7,7), strides = 2
  X = Conv2D(filters = 64, kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu')(input_layer)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # convolutional layer: filters = 64, strides = 1
  X = Conv2D(filters = 64, kernel_size = (1,1), strides = 1, padding = 'same', activation = 'relu')(X)

  # convolutional layer: filters = 192, kernel_size = (3,3)
  X = Conv2D(filters = 192, kernel_size = (3,3), padding = 'same', activation = 'relu')(X)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 1st Inception block
  X = Inception_block(X, f1 = 64, f2_conv1 = 96, f2_conv3 = 128, f3_conv1 = 16, f3_conv5 = 32, f4 = 32)

  # 2nd Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 192, f3_conv1 = 32, f3_conv5 = 96, f4 = 64)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 3rd Inception block
  X = Inception_block(X, f1 = 192, f2_conv1 = 96, f2_conv3 = 208, f3_conv1 = 16, f3_conv5 = 48, f4 = 64)

  # Extra network 1:
  X1 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X1 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X1)
  X1 = Flatten()(X1)
  X1 = Dense(1024, activation = 'relu')(X1)
  X1 = Dropout(0.7)(X1)
  X1 = Dense(2, activation = 'sigmoid')(X1)

  
  # 4th Inception block
  X = Inception_block(X, f1 = 160, f2_conv1 = 112, f2_conv3 = 224, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

  # 5th Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 256, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

  # 6th Inception block
  X = Inception_block(X, f1 = 112, f2_conv1 = 144, f2_conv3 = 288, f3_conv1 = 32, f3_conv5 = 64, f4 = 64)

  # Extra network 2:
  X2 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X2 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X2)
  X2 = Flatten()(X2)
  X2 = Dense(1024, activation = 'relu')(X2)
  X2 = Dropout(0.7)(X2)
  X2 = Dense(2, activation = 'sigmoid')(X2)
  
  
  # 7th Inception block
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, 
                      f3_conv5 = 128, f4 = 128)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # 8th Inception block
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, f3_conv5 = 128, f4 = 128)

  # 9th Inception block
  X = Inception_block(X, f1 = 384, f2_conv1 = 192, f2_conv3 = 384, f3_conv1 = 48, f3_conv5 = 128, f4 = 128)

  # Global Average pooling layer 
  X = GlobalAveragePooling2D(name = 'GAPL')(X)

  # Dropoutlayer 
  X = Dropout(0.4)(X)

  # output layer 
  X = Dense(2, activation = 'sigmoid')(X)
  
  # model
  model = Model(input_layer, [X, X1, X2], name = 'GoogLeNet')

  return model

model = GoogLeNet() 
model.summary()

Model: "GoogLeNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_257 (Conv2D) │ (None, 109, 109,  │      9,472 │ input_layer_6[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_58    │ (None, 54, 54,    │          0 │ conv2d_257[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_258 (Conv2D) │ (None, 54, 54,    │      4,160 │ max_pooling2d_58… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_259 (Conv2D) │ (None, 54, 54,    │    110,784 │ conv2d_258[0][0]  │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_59    │ (None, 26, 26,    │          0 │ conv2d_259[0][0]  │
│ (MaxPooling2D)      │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_261 (Conv2D) │ (None, 26, 26,    │     18,528 │ max_pooling2d_59… │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_263 (Conv2D) │ (None, 26, 26,    │      3,088 │ max_pooling2d_59… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_60    │ (None, 26, 26,    │          0 │ max_pooling2d_59… │
│ (MaxPooling2D)      │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_260 (Conv2D) │ (None, 26, 26,    │     12,352 │ max_pooling2d_59… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_262 (Conv2D) │ (None, 26, 26,    │    110,720 │ conv2d_261[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_264 (Conv2D) │ (None, 26, 26,    │     12,832 │ conv2d_263[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_265 (Conv2D) │ (None, 26, 26,    │      6,176 │ max_pooling2d_60… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_39      │ (None, 26, 26,    │          0 │ conv2d_260[0][0], │
│ (Concatenate)       │ 256)              │            │ conv2d_262[0][0], │
│                     │                   │            │ conv2d_264[0][0], │
│                     │                   │            │ conv2d_265[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_267 (Conv2D) │ (None, 26, 26,    │     32,896 │ concatenate_39[0… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_269 (Conv2D) │ (None, 26, 26,    │      8,224 │ concatenate_39[0… │
│                     │ 32)               │            │                 

 Total params: 8,474,422 (32.33 MB)

 Trainable params: 8,474,422 (32.33 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy', 'accuracy', 'accuracy'])
model.fit(x_train, y_train, epochs=10)
aux = model.evaluate(x_test, y_test)

# from sklearn.metrics import classification_report
# import numpy as np
# Y_test = np.argmax(y_test, axis=1)

# print(classification_report(Y_test, answer))
# # Teste 224 Normal

Epoch 1/10


C:\Users\tadio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\optimizers\base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


36/36 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - dense_24_accuracy: 0.6880 - loss: 0.6522
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 33s 920ms/step - dense_24_accuracy: 0.6865 - loss: 0.6395
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 29s 805ms/step - dense_24_accuracy: 0.6981 - loss: 0.6132
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 29s 817ms/step - dense_24_accuracy: 0.7020 - loss: 0.6127
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 30s 819ms/step - dense_24_accuracy: 0.6924 - loss: 0.6238
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 30s 822ms/step - dense_24_accuracy: 0.7147 - loss: 0.5996
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 30s 831ms/step - dense_24_accuracy: 0.6940 - loss: 0.6171
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 30s 831ms/step - dense_24_accuracy: 0.6987 - loss: 0.6148
Epoch 9/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 30s 831ms/step - dense_24_accuracy: 0.6864 - loss: 0.6227
Epoch 10/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 30s 825ms/step - dense_24_accuracy: 0.7018 - loss: 0.6093
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 238ms/step - dense_24_a

In [ ]:
from sklearn.metrics import classification_report
import numpy as np
Y_test = []

answer = model.predict(x_test)

for elem in y_test:
    Y_test.append(elem)
    
answer_2 = np.argmax(answer[0], axis=1)

print(classification_report(Y_test, answer_2))

#print(len(answer[2][0]))

19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 226ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       126
           1       0.78      1.00      0.88       454

    accuracy                           0.78       580
   macro avg       0.39      0.50      0.44       580
weighted avg       0.61      0.78      0.69       580



C:\Users\tadio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\tadio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\tadio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classificati